In [3]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm

In [2]:
def correlation(df, outcome, predictor, covariates=None, normalize_x=False):

    # Drop rows where the outcome is missing
    df = df.dropna(subset=[outcome])
    
    # Normalize the outcome variable to the range [0, 1]
    y = df[outcome]
    y_norm = (y - y.min()) / (y.max() - y.min())
    df = df.copy()  # avoid modifying the original DataFrame
    df[outcome] = y_norm
    
    # Prepare the design matrix for predictors
    X = df[[predictor]].copy()
    
    # Optionally normalize the predictor to the range [0, 1]
    if normalize_x:
        X[predictor] = (X[predictor] - X[predictor].min()) / (X[predictor].max() - X[predictor].min())
    
    # Add any additional covariates if provided
    if covariates is not None and len(covariates) > 0:
        X_cov = df[covariates].copy()
        if normalize_x:
            # Normalize each covariate
            X_cov = (X_cov - X_cov.min()) / (X_cov.max() - X_cov.min())
        X = pd.concat([X, X_cov], axis=1)
    
    # Add constant term for intercept
    X = sm.add_constant(X)
    
    # Outcome variable (normalized)
    y = df[outcome]
    
    # Fit the OLS model
    model = sm.OLS(y, X).fit()
    
    # Extract the beta coefficient, standard error, and p-value for the main predictor
    beta = model.params[predictor]
    se = model.bse[predictor]
    p_value = model.pvalues[predictor]
    
    # Get the 95% confidence interval for the predictor's beta
    ci_lower, ci_upper = model.conf_int().loc[predictor]
    
    return beta, p_value, se, ci_lower, ci_upper, len(y)


In [12]:
def correlation_stand_beta(df, outcome, predictor, covariates=None,
                normalize_x=False, standardize=False, robust_se=False):
    """
    Fit OLS of outcome ~ predictor (+ covariates).
    - outcome is min-max normalized to [0,1] (keeps your original behaviour).
    - If `standardize=True` both outcome and numeric predictors/covariates are z-scored,
      and the function returns a standardized beta for the main predictor.
    - If `normalize_x=True` numeric predictor/covariates are min-max scaled to [0,1].
    - If `robust_se=True` use HC3 robust SEs.
    Returns: dict with beta, se, p, ci, standardized_beta (if requested), R2, N.
    """
    import numpy as np
    import pandas as pd
    import statsmodels.api as sm

    # Drop rows with missing outcome or predictor
    cols_needed = [outcome, predictor] + (covariates or [])
    df2 = df.dropna(subset=cols_needed).copy()
    N = len(df2)

    # Normalize outcome to [0,1] (as in your original)
    y = df2[outcome]
    y_norm = (y - y.min()) / (y.max() - y.min())
    df2[outcome] = y_norm

    # Prepare X (main predictor)
    X = df2[[predictor]].copy()

    # Optionally normalize predictor/covariates to [0,1]
    if normalize_x:
        X[predictor] = (X[predictor] - X[predictor].min()) / (X[predictor].max() - X[predictor].min())

    # Add covariates (assumes numeric; handle categoricals before calling)
    if covariates:
        X_cov = df2[covariates].copy()
        if normalize_x:
            X_cov = (X_cov - X_cov.min()) / (X_cov.max() - X_cov.min())
        X = pd.concat([X, X_cov], axis=1)

    # Standardize (z-score) if requested — this will replace min-max outcome with z-scored outcome
    standardized_beta = None
    if standardize:
        # z-score outcome and numeric X columns
        df_z = df2.copy()
        # z-score outcome
        df_z[outcome] = (df_z[outcome] - df_z[outcome].mean()) / df_z[outcome].std(ddof=0)
        # z-score predictor and numeric covariates
        for col in X.columns:
            df_z[col] = (df_z[col] - df_z[col].mean()) / df_z[col].std(ddof=0)
        X = sm.add_constant(df_z[X.columns])
        y_use = df_z[outcome]
    else:
        X = sm.add_constant(X)
        y_use = df2[outcome]

    # Fit OLS
    model = sm.OLS(y_use, X).fit()
    if robust_se:
        model = model.get_robustcov_results(cov_type='HC3')

    # Extract results for predictor
    beta = model.params[predictor]
    se = model.bse[predictor]
    p_value = model.pvalues[predictor]
    ci_low, ci_high = model.conf_int().loc[predictor].tolist()
    r2 = model.rsquared

    # If standardized was requested, the beta is already standardized
    if standardize:
        standardized_beta = beta

    return {
        'beta': float(beta),
        'se': float(se),
        'p_value': float(p_value),
        'ci_lower': float(ci_low),
        'ci_upper': float(ci_high),
        'standardized_beta': float(standardized_beta) if standardized_beta is not None else None,
        'r2': float(r2),
        'n': int(N)
    }


In [6]:
path = '' # prediction result path
save_path = 'results/new/'+path.split('/')[-2]+'/'
ehr_age_all = pd.read_csv(path+'age_predictions.csv')
actual_age = pd.read_csv('data/ukb_phenoage_nomissing.csv')
print('actual age:', actual_age.shape)
ehr_age_all = ehr_age_all[ehr_age_all.patid.isin(actual_age.eid)]
print('predicted age:', ehr_age_all.shape)
ehr_age_all.rename(columns={'patid' :'eid','predicted':'ClinialAge Gap', 'baseline_age':'Age'}, inplace=True)
# load the 3 type of biomarkers
fi_biomarker = pd.read_csv('data/frailty_index_final.csv')
age_biomarker = pd.read_csv('data/age_biomarker.csv')
ft_biomarker = pd.read_csv('data/functional_traits_final.csv')
save_path = 'results/'+path.split('/')[-2]
if not os.path.exists(save_path):
    os.makedirs(save_path)

actual age: (348451, 14)
predicted age: (320891, 5)


In [7]:
ehr_age_all_analyses = ehr_age_all.merge(fi_biomarker[['eid', 'frailty index']], on='eid', how='left')
ehr_age_all_analyses = ehr_age_all_analyses.merge(age_biomarker, on='eid', how='left')
ehr_age_all_analyses = ehr_age_all_analyses.merge(ft_biomarker, on='eid', how='left')
cov = pd.read_csv('data/covarients_impute.csv').rename(columns={'eid':'patid'})
cov.rename(columns={'patid':'eid'}, inplace=True)
ehr_age_map_all = ehr_age_all_analyses.merge(cov, on='eid', how='left')

In [9]:
cov_features= ['Age','Sex', 'BMI', 'Systolic Blood Pressure', 'HDL Cholesterol',
       'LDL Cholesterol', 'Smoking', 'ethnicity',  'IMD']
analyses_targets = ehr_age_all_analyses.columns[5:]
len(analyses_targets)

27

correlation

In [19]:
corralation_results = []
for i in range(len(analyses_targets)):
    print(analyses_targets[i])
    result = correlation_stand_beta(ehr_age_map_all, analyses_targets[i], 'ClinialAge Gap', cov_features, normalize_x=False, standardize=True)
    result_df = pd.DataFrame(result, index=[analyses_targets[i]])
    corralation_results.append(result_df)
corralation_results_df = pd.concat(corralation_results)

frailty index
Alanine aminotransferase
Albumin
Aspartate aminotransferase
High sensitivity C-reactive protein
Creatinine
Cystatin C
Total bilirubin
Gamma glutamyltransferase
Insulin-like growth factor 1 (IGF-1)
Leukocyte telomere length
Usual walking pace
Body mass index (BMI)
Self-rated health
Facial aging
Hours of sleep
Tiredness
Insomnia
Systolic blood pressure
Diastolic blood pressure
Arterial stiffness index
Heel bone mineral density
Lung function (FEV1) best measure
Hand grip strength (left)
Hand grip strength (right)
Reaction time
Fluid intelligence score


In [ ]:
corralation_results_df
rename_col = {
    'Usual walking pace': "Usual walking (slow) pace",
    'Self-rated health': "Self-rated (poor) health",
    'Facial aging': "(Older) Facial aging",
}

In [25]:

corralation_results_df.replace(rename_col, inplace=True)
corralation_results_df.rename(columns={'standardized_beta':'Beta', 'p_value':'P-value','se':'SE' ,'ci_lower':'CI Lower', 'ci_upper':'CI Upper', 'n':'N'}, inplace=True)
corralation_results_df['Biomarker'] = corralation_results_df.index
corralation_results_df = corralation_results_df[['Biomarker', 'Beta', 'P-value', 'SE', 'CI Lower','CI Upper', 'N']]
corralation_results_df.to_csv('results/revision/correlation_results_biomarkers_adjusted_covariates_Standard_beta.csv', index=False)